1.SPLIT ORTHO IN TIF TILES

In [ ]:
import rasterio
from rasterio.windows import Window
from rasterio.transform import array_bounds
from rasterio.warp import transform_geom
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from pathlib import Path
import shutil
import numpy as np
import json  # Standard library for GeoJSON
from shapely.geometry import shape, box, mapping
from shapely.ops import transform
import pyproj  # For CRS transformations

# Input directories
orthophoto_folder = Path("/media/bioeos/E/drone/serge_ortho_tif")
cropped_ortho_folder = Path("/home/bioeos/Documents/project_hub/segment_upscaling/output/cropped_ortho")

# GeoJSON file containing filtering geometries
geojson_path = Path("/home/bioeos/Documents/project_hub/segment_upscaling/emprise_lagoon.geojson")

# Tile size
TILE_SIZE = 512
NUM_WORKERS = max(1, cpu_count() - 2)  # Use available CPU cores, leaving some free

# Overlap Settings
HORIZONTAL_OVERLAP = 0.5
VERTICAL_OVERLAP = 0.5

# Compute step size based on overlap
HORIZONTAL_STEP = int(TILE_SIZE * (1 - HORIZONTAL_OVERLAP))
VERTICAL_STEP = int(TILE_SIZE * (1 - VERTICAL_OVERLAP))

# Ensure output directory exists
if cropped_ortho_folder.exists():
    shutil.rmtree(cropped_ortho_folder)
cropped_ortho_folder.mkdir(exist_ok=True, parents=True)

# Load GeoJSON geometries using `geojson` package
with open(geojson_path, "r") as f:
    geojson_data = json.load(f)

geojson_crs = "EPSG:4326"  # Defaulting to WGS84 (likely for GeoJSON), but we will check

# Function to process a single tile
def process_tile(args):
    orthophoto_path, tile_x, tile_y = args
    orthophoto_path = Path(orthophoto_path)
    orthoname = orthophoto_path.stem.replace("_ortho", "")
    tile_output_folder = Path(cropped_ortho_folder, orthoname)
    tile_output_folder.mkdir(exist_ok=True, parents=True)

    with rasterio.open(orthophoto_path) as ortho:
        raster_crs = ortho.crs.to_string()  # Get CRS of the raster
        
        # Reproject GeoJSON geometries to match raster CRS
        project = pyproj.Transformer.from_crs(geojson_crs, raster_crs, always_xy=True).transform
        geoms = [transform(project, shape(feature["geometry"])) for feature in geojson_data["features"]]
        
        window = Window(tile_x, tile_y, TILE_SIZE, TILE_SIZE)
        tile_transform = rasterio.windows.transform(window, ortho.transform)

        # Check if tile intersects with reprojected GeoJSON geometries
        tile_bounds = box(*array_bounds(TILE_SIZE, TILE_SIZE, tile_transform))
        if not any(tile_bounds.within(geom) for geom in geoms):
            return  # Skip tile if it does not overlap with the area

        # Read raster data
        tile_ortho = ortho.read(window=window)

        # Apply threshold to filter out mostly black or white tiles
        greyscale_tile = np.sum(tile_ortho, axis=0) / 3
        
        # Black threshold.
        percentage_black_pixel = np.sum(greyscale_tile == 0) * 100 / TILE_SIZE**2
        if percentage_black_pixel > 5:
            return

        # White threshold.
        percentage_white_pixel = np.sum(greyscale_tile == 255) * 100 / TILE_SIZE**2
        if percentage_white_pixel > 10:
            return        

        tile_filename = f"{orthoname}_{tile_x}_{tile_y}.tif"
        tile_output_path = Path(tile_output_folder, tile_filename)

        tile_meta = ortho.meta.copy()
        tile_meta.update({
            "height": TILE_SIZE,
            "width": TILE_SIZE,
            "transform": tile_transform
        })

        with rasterio.open(tile_output_path, "w", **tile_meta) as dest:
            dest.write(tile_ortho)

# Get all orthophoto files
orthophoto_files = [a for a in sorted(list(orthophoto_folder.iterdir())) if a.suffix == ".tif" and a.is_file()]
# Process each orthophoto
for orthophoto_path in orthophoto_files[:15]:
    with rasterio.open(orthophoto_path) as ortho:

        tile_coords = [(orthophoto_path, x, y) 
                       for x in range(0, ortho.width - TILE_SIZE + 1, HORIZONTAL_STEP) 
                       for y in range(0, ortho.height - TILE_SIZE + 1, VERTICAL_STEP)]
        
        with Pool(NUM_WORKERS) as pool:
            list(tqdm(pool.imap_unordered(process_tile, tile_coords), total=len(tile_coords), desc=f"Processing {orthophoto_path}"))

print("\nProcessing complete. All tiles saved.")


2.CONVERT TIF TILES INTO PNG TILES

In [ ]:
from osgeo import gdal
import numpy as np
from PIL import Image
from pathlib import Path
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

cropped_ortho_folder = Path("/home/bioeos/Documents/project_hub/segment_upscaling/output/cropped_ortho")
cropped_ortho_png_folder = Path("/home/bioeos/Documents/project_hub/segment_upscaling/output/crooped_ortho_png")
if cropped_ortho_png_folder.exists():
    shutil.rmtree(cropped_ortho_png_folder)
cropped_ortho_png_folder.mkdir(exist_ok=True, parents=True)

# Function to convert TIFF to PNG and save in the correct folder
def convert_tiff_to_png(input_dir: Path, output_dir: Path):
    for filepath in tqdm(input_dir.iterdir(), desc=f"Processing {input_dir.name}"):
        png_output_path = Path(output_dir, f'{filepath.stem}.png')

        # Open the input TIFF file
        src_ds = gdal.Open(str(filepath))
        if src_ds:
            # Read the raster data as an array
            raster_data = src_ds.ReadAsArray()

            # Convert to PNG format
            if raster_data.ndim == 3:
                image = Image.fromarray(np.transpose(raster_data[:3], (1, 2, 0)).astype(np.uint8), mode="RGB")
            else:
                raise ValueError(f"Unexpected image format: {raster_data.shape}")

            # Save image as PNG
            image.save(png_output_path)
            # print(f"Converted {filepath.name} to PNG and saved to {output_dir}")

# Function to handle a session of conversion
def process_session(session: Path):
    session_output = Path(cropped_ortho_png_folder, session.name)
    session_output.mkdir(exist_ok=True)
    convert_tiff_to_png(session, session_output)

# Convert orthophoto TIFFs to PNG concurrently
print("Converting cropped orthophoto TIFFs to PNG...")

# Use ThreadPoolExecutor to parallelize session processing
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(process_session, cropped_ortho_folder.iterdir()), desc="Sessions"))

print("All conversions completed.")
